<a href="https://www.bigdatauniversity.com"><img src = "https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width = 400, align = "center"></a>

<h1 align=center><font size = 5>Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto-</font></h1>

In [13]:
# Libraries:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
#import folium # map rendering library

# import request to get web page:
import requests

print('Libraries imported.')

Libraries imported.


In [10]:
# Get Wikipedia page:
wikipedia_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki = requests.get(wikipedia_link)
page = wiki.text
#print(page)
del(wiki)
# Find line that table begin and end:
# Table start: <table class="wikitable sortable">
# Table end: </tbody></table>
table_begin = page.find('<table class="wikitable sortable">')
table_end = page.find('</tbody></table>')
#print (table_begin , table_end)

# Read webpage table, and set first table row as columns header:
toronto = pd.read_html(page[table_begin:table_end], header=0)[0]
#print (toronto.head(5))

# Filter 'Not assigned'
toronto_filter = toronto[toronto['Borough'] != 'Not assigned']
del(toronto)
#print ('\n\n\n')
#print (toronto_filter.head(5))

# Filter neighbourhood with Not assigned, and replace to Borought
toronto_filter.loc[toronto_filter.Neighbourhood == 'Not assigned' , ['Neighbourhood']] = toronto_filter['Borough']
#print(toronto_filter.head(11))

# Concatenate Neighboruhood with same Postcode and Borough:
toronto_group = toronto_filter.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

print ('shape: ',toronto_group.shape)
toronto_group.head(15)

shape:  (103, 3)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


# Geocoder - Find location

Try with 2 different library, but it didn't work. 
In the end make with CSV file.

In [25]:
# Try with Nominatim, but query with PostCode is `experimental`, and only works with complet Postcode
# The information in the table look like is missing a second part.

### 
# from: https://wiki.openstreetmap.org/wiki/Nominatim
# postalcode=<postalcode>

# (experimental) Alternative query string format for structured requests.
# Structured requests are faster and require fewer server resources.
# For North American addresses, use the full name for state to improve the results.
# DO NOT COMBINE WITH q=<query> PARAMETER.

# for postalcode= M5G 202J3
# Works: https://nominatim.openstreetmap.org/search?postalcode=M5G%202J3&format=xml&polygon=0&addressdetails=1

#without the second part didn`t work.

'''
address = 'city=Toronto,state=Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print ('lat: ' , latitude , 'long: ', longitude)
'''

'\naddress = \'city=Toronto,state=Ontario\'\n\ngeolocator = Nominatim(user_agent="toronto_explorer")\nlocation = geolocator.geocode(address)\nlatitude = location.latitude\nlongitude = location.longitude\n\nprint (\'lat: \' , latitude , \'long: \', longitude)\n'

In [26]:
'''
# !conda install -c conda-forge geocoder --yes # 
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None
postal_code='M5G'
# loop until you get the coordinates
i = 0
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng
  i = i + 1 
  if i > 10:
        print ('erro!!!')
        exit()

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

print (latitude, ' long: ', longitude)
'''

"\n# !conda install -c conda-forge geocoder --yes # \nimport geocoder # import geocoder\n\n# initialize your variable to None\nlat_lng_coords = None\npostal_code='M5G'\n# loop until you get the coordinates\ni = 0\nwhile(lat_lng_coords is None):\n  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))\n  lat_lng_coords = g.latlng\n  i = i + 1 \n  if i > 10:\n        print ('erro!!!')\n        exit()\n\nlatitude = lat_lng_coords[0]\nlongitude = lat_lng_coords[1]\n\nprint (latitude, ' long: ', longitude)\n"

In [27]:
# Use CSV file provide in exercise
#!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

df_geodata = pd.read_csv('Geospatial_Coordinates.csv')
print (df_geodata.head(5))

Data downloaded!
  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476


In [28]:
# join geodata with toronto dataframe
toronto = toronto_group.set_index('Postcode').join(df_geodata.set_index('Postal Code'))
print('shape : ' , toronto.shape)
toronto

shape :  (103, 4)


,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
